In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Flatten, Conv1D, MaxPool1D

In [5]:
def mlp(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        hidden_layers=[32, 16, 8],
        dropout=0.0):
    
    # 2d input suitable for CNN and LSTM
    inputs = Input(shape=(input_shape[-2], input_shape[-1]))
    # Flatten 2d input into 1d
    x = Flatten()(inputs)
    # Add Dense + Dropout for every hidden layer
    for hidden_units in hidden_layers:
        x = Dense(hidden_units)(x)
        x = Dropout(dropout)(x)
    # Output layer to predict {horizon} time steps ahead
    x = Dense(horizon)(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(optimizer=optimizer, loss=loss)

    return model

In [ ]:
def cnn(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        conv_blocks=[[64, 7, 2], [128, 5, 2]]
        dense_layers=[],
        dense_dropout=0.0):
    
    conv_layers = [b[0] for b in conv_blocks]
    kernels = [b[1] for b in conv_blocks]
    pool_sizes = [b[2] for b in conv_blocks]
    
    inputs = Input(shape=(input_shape[-2], input_shape[-1]))
    
    x = Conv1D(conv_layers[0], kernels[0], activation="relu", padding="same")(inputs)
    
    if pool_sizes[0]:
        x = MaxPool1D(pool_size=pool_sizes[0])(x)
    
    for chanels, kernel, pool_size in zip(conv_layers[1:], kernels[1:], pool_sizes[1:]):
        x = Conv1D(chanels, kernel, activation="relu", padding="same")(x)
        if pool_size:
            x = MaxPool1D(pool_size=pool_size)(x)
    # Dense block
    x = Flatten()(x)
    for hidden_units in dense_layers:
        x = Dense(hidden_units)(x)
        x = Dropout(dense_dropout)(x)
    x = Dense(horizon)(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(optimizer=optimizer, loss=loss)
    
    return model

In [10]:
def lstm(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        lstm_layers=[50],
        lstm_dropout=0.0,
        return_sequences=False,
        dense_layers=[],
        dense_dropout=0.0):
    
    inputs = Input(shape=(input_shape[-2], input_shape[-1]))
    
    return_seq = return_sequences if len(lstm_layers) == 1 else True
    x = LSTM(lstm_layers[0], return_sequences=return_seq, dropout=lstm_dropout)(inputs)
    
    for i, u in enumerate(lstm_layers[1:]):
        return_seq = return_sequences if i == len(lstm_layers) - 2 else True
        
        x = LSTM(u, return_sequences=return_seq, dropout=lstm_dropout)(x)
    
    if return_sequences:
        x = Flatten()(x)
    
    for units in dense_layers:
        x = Dense(units)(x)
        x = Dropout(dense_dropout)(x)
    
    x = Dense(horizon)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(optimizer=optimizer, loss=loss)
    
    return model

In [14]:
model = lstm((10,3,3), lstm_layers=[50,25,10], horizon=5, lstm_dropout=0.2, dense_layers=[100, 25], dense_dropout=0.2)
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3, 3)]            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 3, 50)             10800     
_________________________________________________________________
lstm_10 (LSTM)               (None, 3, 25)             7600      
_________________________________________________________________
lstm_11 (LSTM)               (None, 10)                1440      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               1100      
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 25)               